# LangChain

(deels geïnspireerd door een lessenreeks van DeepLearning.ai)

Voor we starten gaan we eerst zorgen dat iedereen een werkende setup heeft. Als je vorige keer OpenAI opgezet hebt, en je api key staat nog altijd mooi in de `.env` dan is alles oké.
Als je credits op waren, is er een alternatief via [Eden AI](https://www.edenai.co/), Eden AI heeft als business model om via één centrale API (de hunne) allerlei andere AI API's aan te spreken. Maar wat voor ons vooral handig is, is dat je een account kan aanmaken zonder credit card, en dan $10 gratis credits krijgt (en dus kunnen we via hen OpenAI aanspreken)

Ga naar [www.edenai.co](https://www.edenai.co/) en maak een account aan.

<div>
    <img src="img/edenai_signup.png" style="width: 700px;" >
</div>


Dan kan je bij Account Management je API key vinden

<div>
    <img src="img/edenai_apikey.png" style="width: 700px;" >
</div>



Bewaar nu deze API key ook in je `.env` file (de .env file moet ergens in een parent folder van al je notebooks staan, dus als je de repo gewoon gecloned hebt gewoon in de `2324-trendsinai/` folder zelf

```bash
OPENAI_API_KEY=sk-YYYYY
EDENAI_API_KEY=XXXXXXXX
```

In [ ]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key = os.getenv('OPENAI_API_KEY')
edenai_api_key = os.getenv('EDENAI_API_KEY')

# print (openai.api_key)
# print (edenai_api_key)


Als je de print commando's uitvoert, zou je de key(s) die je wilt gebruiken moeten zien, als je `None` ziet staan is er iets mis.

De key van EdenAI is een Bearer token, daar kennen jullie alles van uit de Webservices cursus. Rechtstreeks de EdenAI API aanspreken doe je met een POST request naar de juiste url.
EdenAI heeft véél endpoints, een ChatGPT request zou er als volgt uitzien:

In [ ]:
import requests

url = "https://api.edenai.run/v2/text/chat"

payload = {
    "response_as_dict": True,
    "attributes_as_list": False,
    "show_original_response": False,
    "temperature": 0,
    "max_tokens": 1000,
    "providers": "openai",
    "text": "what are the advantages of langchain?"
}
headers = {
    "accept": "application/json",
    "content-type": "application/json",
    "authorization": f"Bearer {edenai_api_key}"
}

response = requests.post(url, json=payload, headers=headers)

print(response.text)

We krijgen een string waar ogenschijnlijk wel een JSON-object in zit met key `openai` (we kunnen dit ook direct naar meerdere LLM's sturen en dan zien we elke response), een `status` (hopelijk success) en dan de `generated_text`, er volgt ook nog een array met de role-user, role-assistant historiek in het geval van openai, en op het einde een `cost`.

Toen ik deze vraag stelde, kostte het antwoord mij 0.000708; dus met onze 10 dollar kunnen we een 15.000 van dat soort requests sturen, veel, maar ook niet oneindig.

We moeten dit JSON-object wel eerst omzetten naar Python, voor we verder kunnen.

In [ ]:
import json
from types import SimpleNamespace

data = '{"openai":{"status":"success","generated_text":"Example of generated_text" }}'

# Parse JSON into an object with attributes corresponding to dict keys.
x = json.loads(data, object_hook=lambda d: SimpleNamespace(**d))
print(x.openai.generated_text)

Als we nu alles samenbrengen kunnen we onze `get_answer` van vorige keer aanpassen om of OpenAI, of EdenAI of GPT4All te gebruiken al naargelang.

In [ ]:
from gpt4all import GPT4All
model = GPT4All("ggml-model-gpt4all-falcon-q4_0.bin")

In [ ]:
from enum import Enum
import json
from types import SimpleNamespace

class API(Enum):
    OPEN_AI = 1
    GPT4All = 2
    EDEN_AI = 3


def get_answer(prompt, which_model=API.OPEN_AI):
    if which_model == API.GPT4All:
        res = model.generate(prompt)
        return res
    elif which_model == API.OPEN_AI:
        message = [{"role": "user", "content": prompt}]
        res = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=message
        )
        return res.choices[0].message["content"]
    elif which_model == API.EDEN_AI:
        url = "https://api.edenai.run/v2/text/chat"

        payload = {
            "response_as_dict": True,
            "attributes_as_list": False,
            "show_original_response": False,
            "temperature": 0,
            "max_tokens": 1000,
            "providers": "openai",
            "text": f"{prompt}"
        }
        headers = {
            "accept": "application/json",
            "content-type": "application/json",
            "authorization": f"Bearer {edenai_api_key}"
        }

        response = requests.post(url, json=payload, headers=headers)
        responseObject = json.loads(response.text, object_hook=lambda d: SimpleNamespace(**d))
        return responseObject.openai.generated_text;

            

In [ ]:
print("GPT4All")
print(get_answer("What are the advantages of GPT4All", API.GPT4All))
print("Open AI")
print(get_answer("What are the advantages of OpenAI", API.OPEN_AI))
print("Eden AI")
print(get_answer("What are the advantages of EdenAI", API.EDEN_AI))

## LangChain, waarom?

LangChain is een framework dat dient om applicaties die gebruik maken van language models te ontwikkelen. Het laat toe om vrij eenvoudig verschillende componenten aan elkaar te 'chainen' (vandaar de naam), en zo meer gestandaardiseerd allerlei verschillende modellen te gebruiken.

Het is zeer snel, zeer groot geworden (al meer dan 1700 contributors op [github](https://github.com/langchain-ai/langchain) voor de python versie, en nog eens 400 voor de js versie)
Het leuke is dat je snel veel soorten AI modellen kan gebruiken, of makkelijk uitbreiden met vectorstores en andere extra functionaliteit; en je bij een wissel naar een ander model niet per se alles opnieuw dient te programmeren of aan te passen.

Een (voorlopig?) nadeel is dat het zo snel evolueert en groeit dat de documentatie best wel te wensen over laat.  

## Models

We gaan eerst een llm model creëeren, je hebt de keuze tussen een EdenAI of OpenAI model.

## Prompts

Om makkelijker prompts te genereren die (deels) variabel zijn heeft LangChain [PromptTemplates](https://python.langchain.com/docs/modules/model_io/prompts/prompt_templates/), een PromptTemplate krijgt een string met {variabelen} als `template`, en genereert dan samen met een array van `input_variables` een prompt die kan gebruikt worden voor een llm.



## Output formatting

De standaard output is gewoon een stuk tekst. Vaak wil je echter JSON of een ander gestructureerd formaat om dan makkelijker verder te kunnen werken (en 'chainen')

Als voorbeeld nemen we een review van Disneyland, genomen uit een dataset van [https://www.kaggle.com/](https://www.kaggle.com/)

In [ ]:
review = f"""If you've ever been to Disneyland anywhere you'll find Disneyland Hong Kong very similar in the layout when you walk into main street! It has a very familiar feel. One of the rides  its a Small World  is absolutely fabulous and worth doing. The day we visited was fairly hot and relatively busy but the queues moved fairly well."""


review_template = """\
For the following text, extract the following information:

location: Was the review about Disneyland Paris, California, Hong Kong or Unknown

weather: Was there any indication about the weather conditions, answer with \
'TOO HOT', 'HOT', 'RAIN', 'COLD' or 'UNKNOWN' if no information was provided

rides: Extract any sentences about the rides,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
location
weather
rides

text: {text}
"""


Zoals je kan zien krijgen we een `string` terug van het systeem, het zou natuurlijk veel beter zijn als dit een echt JSON object is. We gebruiken hiervoor [StructuredOutputParser](https://python.langchain.com/docs/modules/model_io/output_parsers/structured)

## Indexes

### Conversation Memory

Met behulp van een [https://python.langchain.com/docs/modules/memory/types/buffer](Conversation Buffer) krijgt je conversatie een 'geheugen', je hoeft niet langer de prompts in hun geheel mee te geven. (standaard start elke prompt van nul, en kan de AI dus geen rekening houden met wat er reeds eerder gezegd is)

Je kan de grootte van je conversation buffer zelf beheren.

In plaats van de buffer te beperken op basis van het aantal vraag-antwoorden, kan je het ook op basis van het aantal tokens doen met behulp van `ConversationTokenBufferMemory`

### oefening 

Gebruik TokenBufferMemory om een deel van een conversatie te beperken, kijk wat er gebeurt als de 'helft' van een zin wegvalt door de limit


Een `ConversationSummaryBufferMemory` gaat niet gewoon alles uit het geheugen gooien als er geen plaats meer is (en alles waarvoor er wel plaats is letterlijk onthouden) maar gaat proberen een samenvatting te maken van wat anders zou verdwijnen.

## Chains

Lang*Chain*, er bestaan een heleboel 'Chain' klassen die je toestaan de output van één prompt als input voor de volgende te gebruiken. 

Bij een `SimpleSequentialChain` wordt de output van de ene als input voor de volgende gebruikt, zonder dat er nood is aan een key of iets dergelijks. Het systeem gaat er vanuit dat de output van de ene de (enige) input van de andere is.

Als er iets complexere kettingen dienen gevormd te worden, waar een prompt input dient te krijgen van twee verschillende andere prompts bijvoorbeeld, gebruiken we een `SequentialChain`, hier kan je telkens een key associëren met een output, en zo meer controle krijgen over welke output bij welke input gebruikt wordt. 

## RetrievalQA: Q&A over documenten

We hebben al heel wat klassen van langchain de revue zien passeren en kleinere chains gebouwd. Tijd om eens een wat 'echtere' toepassingen na te bootsen.
Nu gaan we een grote(re) hoeveelheid data inladen gebruik makend van een vectorstore; en dan een Q&A systeem over deze data creëeren.

Als voorbeeld nemen we een dataset van Steam games, bekijk eerst de data even in een terminal (of iets dergelijks). 
 

`Embeddings` zorgen ervoor dat we een vector representatie van een stuk tekst kunnen genereren., 

We kunnen nu similarity searches uitvoeren gebruik makend van vectorstores. Een makkelijk te hanteren versie is de `DocArrayInMemorySearch`. Zoals de naam suggereert gebeurt alles in memory, dus enkel geschikt als de data niet té groot is, maar heeft het grote voordeel dat we geen externe vector store API's moeten aanspreken of configureren.

Zo wordt er wel een gelijkaardig resultaat gevonden, op basis van de vector embedding, maar om wat 'menselijkere' output te krijgen, in de vorm van vraag-antwoord, gebruiken we de `RetrievalQA`

Die zal een llm combineren met onze vectordatabank.

Er bestaat ook een makkelijkere manier om hetzelfde te bekomen, een `VectorStoreIndexCreator`, die veel korter juist hetzelfde doet.
Alles nog eens opnieuw (dus ook document loaden e.d., om te tonen hoe weinig code nodig is)

## Evalueren

Hoe kunnen we nu zien hoe goed deze chains hun werk doen? We zouden manueel kunnen door onze data gaan en goede voorbeelden vinden, daar vragen op loslaten, en kijken of de antwoorden voldoen.
Maar dat schaalt natuurlijk absoluut niet. Dus we gaan een llm chain gebruiken om onze llm chain te evalueren.



Dan kunnen we deze vragen uitvoeren en zien of de antwoorden correct gegeven worden, maar manueel schaalt dat natuurlijk weer niet zo goed.

Dus we gaan al deze vragen door QAChain sturen 

En dan kunnen we ze allemaal tegelijk laten evalueren met behulp van een QAEvalChain


## Agents

Agents zijn een vrij nieuwe toevoeging aan LangChain, en laten toe om andere soorten externe bronnen toe te voegen. Bijvoorbeeld een rekenmodule, wikipedia of een zoekmachine.